In [5]:
import requests
import os
from typing import List, Optional, Dict



SERPAPI_API_KEY = ''

def get_event_reviews(event_name: str, location: Optional[str] = None, limit: int = 7) -> List[str]:
    """
    Extract top review texts for an event using SerpAPI.
    
    Args:
        event_name: Name of the event
        location: Optional location of the event
        limit: Maximum number of reviews to return (default: 7)
        
    Returns:
        List of review text snippets
    """
    api_key = SERPAPI_API_KEY
    
    if not api_key:
        raise ValueError(
            "SerpAPI key is required. Set the SERPAPI_API_KEY environment variable."
        )
    
    # Construct the query
    query = event_name
    if location:
        query += f" {location}"
    query += " reviews experiences"
    
    # Set up the SerpAPI parameters
    params = {
        "api_key": api_key,
        "engine": "google",
        "q": query,
        "num": 10,  # Request more than needed to ensure we get enough usable snippets
        "gl": "us",  # Country (Google locality)
        "hl": "en"   # Language
    }
    
    try:
        # Send the request to SerpAPI
        response = requests.get("https://serpapi.com/search", params=params)
        
        if response.status_code != 200:
            return [f"Error fetching reviews: HTTP {response.status_code}"]
        
        data = response.json()
        reviews = []
        
        # Extract organic results snippets
        if "organic_results" in data:
            for result in data["organic_results"]:
                if "snippet" in result and result["snippet"]:
                    reviews.append(result["snippet"])
                    
                    # Break if we have enough reviews
                    if len(reviews) >= limit:
                        break
        
        # If we still need more reviews, try extracting from "people also ask"
        if len(reviews) < limit and "related_questions" in data:
            for question in data["related_questions"]:
                if "snippet" in question and question["snippet"]:
                    reviews.append(question["snippet"])
                    
                    # Break if we have enough reviews
                    if len(reviews) >= limit:
                        break
        
        return reviews[:limit]  # Ensure we don't return more than the limit
        
    except Exception as e:
        return [f"Error: {str(e)}"]

def format_reviews_for_llm(event_name: str, location: Optional[str] = None, limit: int = 7) -> str:
    """
    Get reviews and format them for sending to another LLM.
    
    Args:
        event_name: Name of the event
        location: Optional location of the event
        limit: Maximum number of reviews to return
        
    Returns:
        Formatted string with reviews
    """
    reviews = get_event_reviews(event_name, location, limit)
    
    if not reviews:
        return f"No reviews found for {event_name}"
    
    # If the first review starts with "Error", it's an error message
    if reviews[0].startswith("Error"):
        return reviews[0]
    
    # Format the reviews
    event_info = event_name
    if location:
        event_info += f" in {location}"
        
    formatted = f"Reviews for {event_info}:\n\n"
    
    for i, review in enumerate(reviews, 1):
        formatted += f"Review #{i}: {review}\n\n"
    
    return formatted


if __name__ == "__main__":
    event_name = "Give me review about $1 oysters at publico street bistro & garden"
    location = "Boston"
    
    formatted_reviews = format_reviews_for_llm(event_name, location, limit=5)
    print(formatted_reviews)

Reviews for Give me review about $1 oysters at publico street bistro & garden in Boston:

Review #1: The food was also very good. Nice location with an open layout. ... Photo of Janice C. ... Janice C. ... On Tuesdays, it's $1.00 oysters! I have never tried oysters.

Review #2: This place is definitely filled with more of a younger crowd but ambiance is lit. Music is turned up and eating here is fun. We had a full course here and ...

Review #3: Publico is a great spot! Good vibes, friendly. Food is on the higher end of your typical bar food. And the chicken wings are amazing!

Review #4: Publico Street Bistro & Garden ... South Boston restaurant and bar Publico is serving up $1 oysters — and $1 shrimp cocktail — every Monday and ...

Review #5: Mondays & Tuesdays are officially the best days of the week, with fresh oysters & shrimp at $1 each! Trust us, they go FAST, so get here early, grab a drink, ...


